<a href="https://colab.research.google.com/github/BrokenShell/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [x] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [x] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [x] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import itertools as it

In [0]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Data Clean Up
df = df.drop(columns=['latitude', 'longitude', 'description', 'display_address', 'street_address'])
df['interest_level'] = df['interest_level'].apply(lambda itm: {'low': 1, 'medium': 2, 'high': 3}[itm])

In [0]:
df['created'] = df['created'].apply(lambda date: int(date.split('-')[1]))

In [0]:
df.head()

,bathrooms,bedrooms,created,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,6,3000,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,6,5465,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,4,2850,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,4,3275,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,4,3350,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Split Train & Test for [April & May] vs June
train, test = df[df['created'] < 6], df[df['created'] == 6]
X_train_base, y_train = train.drop(columns=(['price', 'created'])), train['price']
X_test_base, y_test = test.drop(columns=(['price', 'created'])), test['price']

In [0]:
# Baseline Mean
guess = y_train.mean()
print(f"Baseline Average: ${guess:.2f}")

Baseline Average: $3575.60


In [0]:
# Baseline Training Error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f"Baseline Training Error: ${mae:.2f}")

Baseline Training Error: $1201.88


In [0]:
# Baseline Testing Error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Baseline Testing Error: ${mae:.2f}")

Baseline Testing Error: $1197.71


## Engineering Features & Linear Regression

In [0]:
def find_mae(features):
    model = LinearRegression()
    # print(*features)

    X_train = X_train_base[features]
    X_test = X_test_base[features]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred)
    # print(f"Training Error: {mae:.2f}")

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    # print(f"Testing Error: {mae:.2f}\n")
    return mae, features, model.intercept_, model.coef_

In [0]:
def find_rmse(features):
    model = LinearRegression()
    X_train = X_train_base[features]
    X_test = X_test_base[features]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return np.sqrt(mse)

In [0]:
def find_r2_score(features):
    model = LinearRegression()
    X_train = X_train_base[features]
    X_test = X_test_base[features]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

### Regress All The Things!!!

In [0]:
features = [
    'bathrooms', 'bedrooms', 'interest_level', 'hardwood_floors', 
    'dogs_allowed', 'doorman', "dishwasher", "no_fee", 
    "laundry_in_building", "fitness_center", "pre-war", 
    "laundry_in_unit", "roof_deck",	"outdoor_space", 
    "dining_room", "high_speed_internet", "balcony", 
    "swimming_pool", "new_construction", "terrace", 
    "exclusive", "loft", "garden_patio", "wheelchair_access",
    "common_outdoor_space"
]
mae, _, intercept, coef = find_mae(features)
print(f"Test Mean Absolute Error: ${mae:.2f}")
rmse = find_rmse(features)
print(f"Test Root Mean Squared Error: ${rmse:.2f}")
r2 = find_r2_score(features)
print(f"Test R2 Score: {r2:.5f}\n")
print(f"MAE Intercept: ${intercept:.2f}\n")
print("Unscaled MAE Coefficients:")
for f, c in zip(features, coef):
    print(f"{f}: {c:.2f}")

Test Mean Absolute Error: $729.41
Test Root Mean Squared Error: $1102.03
Test R2 Score: 0.60925

MAE Intercept: $1034.31

Unscaled MAE Coefficients:
bathrooms: 1706.73
bedrooms: 470.43
interest_level: -438.25
hardwood_floors: -109.46
dogs_allowed: 72.39
doorman: 625.77
dishwasher: 133.91
no_fee: -117.47
laundry_in_building: -60.21
fitness_center: 209.75
pre-war: -88.12
laundry_in_unit: 528.10
roof_deck: -138.62
outdoor_space: -87.90
dining_room: 186.81
high_speed_internet: -270.28
balcony: -64.76
swimming_pool: 12.66
new_construction: -128.58
terrace: 226.79
exclusive: 132.90
loft: 235.16
garden_patio: -53.02
wheelchair_access: 146.10
common_outdoor_space: -7.78


Due to the fact that our inputs are not scaled and they have different units, the coefficients above are not comparable.

In [0]:
def best_n_features(n_features):
    min_error = np.inf
    for feat in it.combinations(features, n_features):
        error, feats, _, _ = find_mae(list(feat))
        if error < min_error:
            min_error = error
    print(feats, f"${min_error:.2f}")

### Best Single Feature

In [0]:
best_n_features(1)

['common_outdoor_space'] $890.46


### Best 2 Features

In [0]:
best_n_features(2)

['wheelchair_access', 'common_outdoor_space'] $825.90


### Best 3 Features

In [0]:
best_n_features(3)

['garden_patio', 'wheelchair_access', 'common_outdoor_space'] $776.35


### Best 4 Features

In [0]:
best_n_features(4)

['loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space'] $741.91


### Best 5 Features

In [0]:
best_n_features(5)

['exclusive', 'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space'] $736.66


## Summary
Seems to be slowly approaching $729.41 as the number of features goes up.

## Personal Stretch Goal: Command Line Predictions

In [0]:
!pip install MultiChoice

In [0]:
import MultiChoice as mc

In [0]:
model = LinearRegression()
features = ['bathrooms', 'bedrooms']
X_train = X_train_base[features]
X_test = X_test_base[features]

model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f"Training Error: {mae:.2f}")

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Testing Error: {mae:.2f}")

Training Error: 818.53
Testing Error: 825.90


In [0]:
def cli_predict():
    bedrooms = mc.Question("How many Bedrooms?")()
    bathrooms = mc.Question("How many Bathrooms?")()
    prediction = model.predict([[float(bathrooms), int(bedrooms)]])[0]
    print(f"Predicted Price: ${prediction:.2f} +/- {mae:.0f}")

In [0]:
cli_predict()

How many Bedrooms?
>>> 1
How many Bathrooms?
>>> 0.5
Predicted Price: $1911.35 +/- 826


In [0]:
cli_predict()

How many Bedrooms?
>>> 1
How many Bathrooms?
>>> 1
Predicted Price: $2947.65 +/- 826


In [0]:
cli_predict()

How many Bedrooms?
>>> 1
How many Bathrooms?
>>> 1.5
Predicted Price: $3983.96 +/- 826


In [0]:
cli_predict()

How many Bedrooms?
>>> 2
How many Bathrooms?
>>> 1
Predicted Price: $3336.98 +/- 826


In [0]:
cli_predict()

How many Bedrooms?
>>> 5
How many Bathrooms?
>>> 3.5
Predicted Price: $9686.48 +/- 826
